# À propos de ce livret




*Utilisation d'une méthode de base.*




### Recherches
>Renaud Gaujoux, Cathal Seoighe (2010). **A flexible R package for nonnegative matrix factorization.**  
>Xihui Lin, Paul C. Boutros (2020). [**Optimization and expansion of non-negative matrix factorization.**](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6945623/)

### Sources
>[**Dépôt 1**](https://github.com/linxihui/NNLM) | *Version R*  
>[**Dépôt 2**](https://github.com/timothyliu-datascience/Non-negative-Matrix-Factorization/blob/master/NMFN.R) | *Version R*  




### Note sur le cachier

>[**Exemples d'implémentation**](https://rdrr.io/cran/NNLM/f/inst/doc/Fast-And-Versatile-NMF.pdf)  | *Document explicatif*



## Préparation pour l'utilisation

In [22]:
## load up the packages we will need:  (uncomment as required)

# Installer les paquets s'ils sont manquants 
pac = c("devtools", "R.matlab", "plyr", "ggplot2", "dplyr", "jsonlite")

new_pac <- pac[!(pac %in% installed.packages()[, "Package"])]
if (length(new_pac))
  install.packages(new_pac)

## Chargement des matrices "matlab"
library(R.matlab)

## Manipulation de données
library(plyr)
library(dplyr)

## Transfert des listes vers Python
library(jsonlite)

## Visualisation (dans un autre livret)
# library(ggplot2)

# Données

## Chargement des données

In [23]:
parameters <- list()

## Fichiers
parameters$dossier = "data/transport_Seattle"
parameters$fichier_complet = "/seattle_50"
parameters$fichier_binaire = "/seattle_50_50"
parameters$dossier_experience = "exp/transport_Seattle"
parameters$fichier_experience = "/seattle_50_50_FMNN"


## Entrainement (paramètres)
parameters$rang = 10
parameters$algorithme = "nnmf_mm"
parameters$max_iter = 1000


dossier = parameters$dossier
fichier.complet = parameters$fichier_complet
fichier.binaire = parameters$fichier_binaire

In [24]:
mat.complet <- readMat(paste0(dossier, fichier.complet, ".mat"))
mat.complet <- mat.complet$mat
mat.binaire <- readMat(paste0(dossier, fichier.binaire, ".mat"))
mat.binaire <- mat.binaire$mat

## Création de la matrice de manquants

In [25]:
# Création de la matrice de manquants
manq.mat <- mat.complet * mat.binaire
# Créer un index pour la validation des paramètres imputés
index <- which(mat.binaire %in% c(0))

# Modélisation

### Fonctions d'utilités

In [26]:
# Moore-Penrose Inverse
mpinv <- function(X) {
    Eps <- 100 * .Machine$double.eps
    
    # singular value decomposition
    s <- svd(X)
    d <- s$d
    m <- length(d)
    
    if (!(is.vector(d))) 
        return(t(s$v %*% (1/d) %*% t(s$u)))
    
    d <- d[d > Eps]
    notnull <- length(d)
    
    if (notnull == 1) {
        inv <- 1/d
    } else {
        inv <- solve(diag(d))
    }
    
    if (notnull != m) {
        inv <- cbind(inv, matrix(0, nrow = notnull, ncol = (m - notnull)))
        inv <- rbind(inv, matrix(0, nrow = (m - notnull), ncol = m))
    }
    
    mp <- s$v %*% inv %*% t(s$u)
    
    mp[abs(mp) < Eps] <- 0
    return(mp)
}

In [27]:
# Euclidean Distance between two matrices
distance2 <- function(x1, x2) {
    temp <- x1 - x2
    sum(temp * temp)
}

In [28]:
# Root mean square error (RMSE)
RMSE <- function(val_orig, val_pred, index) {
    ## Comptabilisé sur les valeurs à imputer
    rmse <- sqrt(mean((val_orig[index] - val_pred[index])^2))
    rmse <- format(rmse, digits = 6)
    rmse
}

In [29]:
## Mean absolute percentage error (MAPE)
MAPE <- function(val_orig, val_pred, index) {
    ## Comptabilisé sur les valeurs à imputer
    mape <- sum(abs(val_orig[index] - val_pred[index])/val_orig[index])/length(val_orig[index])
    mape <- format(mape, digits = 6)
    mape
}

## Modèles

In [30]:
# Non-negative Matrix Factorization via alternating least squares
nnmf_als <- function(x, k, maxiter, eps, index, x_complet) {
    print_iter <- 50  # iterations between print
    x <- as.matrix(x)
    
    if (any(!is.finite(x))) 
        stop("infinite or missing values in 'x'")
    
    dx <- dim(x)
    D <- dx[1L]
    N <- dx[2L]
    Xscale = sum(x)
    
    
    if (!D || !N) 
        stop("0 extent dimensions")
    set.seed(2020)
    W <- matrix(abs(rnorm(D * k)), D, k)
    set.seed(2020)
    H <- matrix(abs(rnorm(k * N)), k, N)
    
    Rscale <- sum(W %*% H)
    sqrnorm <- sqrt(Rscale/Xscale)
    
    H <- H/sqrnorm
    W <- W/sqrnorm
    
    Xr_old <- W %*% H
    
    for (iter in 1:maxiter) {
        W <- x %*% t(mpinv(H %*% t(H)) %*% H)
        W <- (W > 0) * W
        W <- W/(t(matrix(rep(colSums(W), D), ncol(W), nrow(W))) + eps)
        H <- t(W %*% mpinv(t(W) %*% W)) %*% x
        H <- H * (H > 0)
        
        if (iter%%print_iter == 0) {
            Xr <- W %*% H
            diff <- sum(abs(Xr_old - Xr))
            Xr_old <- Xr
            
            eucl_dist <- distance2(x, W %*% H)
            errorx <- mean(abs(x - W %*% H))/mean(x)
            x_pred = W %*% H
            rmsex <- sqrt(mean((x_complet[index] - x_pred[index])^2))
            
            cat("Iter = ", iter, "\t")
            cat("relative error = ", errorx, "\t")
            cat("RMSE", rmsex, "\t")
            cat("diff = ", diff, "\t")
            cat("eucl dist = ", eucl_dist, "\n")
            
            if (errorx < 1e-05) {
                cat("Execution finishes at iteration = ", iter, "\n")
                break
            }
        }
    }
    z <- c(list(W = W, H = H))
    z
}

In [31]:
# Non-negative Matrix Factorization via multiplicative update
nnmf_mm <- function(x, k, maxiter, eps, index, x_complet) {
    print_iter <- 50  # iterations between print
    x <- as.matrix(x)
    
    if (any(!is.finite(x))) 
        stop("infinite or missing values in 'x'")
    
    dx <- dim(x)
    n <- dx[1L]
    m <- dx[2L]
    
    if (!n || !m) 
        stop("0 extent dimensions")
    
    set.seed(2020) # Reproductibitlité
    W <- matrix(abs(rnorm(n * k)), n, k)
    set.seed(2020) # Reproductibitlité
    H <- matrix(abs(rnorm(k * m)), k, m)
    
    Xr_old <- W %*% H
    
    for (iter in 1:maxiter) {
        H <- H * (t(W) %*% x)/((t(W) %*% W) %*% H + eps)
        W <- W * t(H %*% t(x))/(W %*% (H %*% t(H)) + eps)
        
        if (iter%%print_iter == 0) {
            Xr <- W %*% H
            diff <- sum(abs(Xr_old - Xr))
            Xr_old <- Xr
            
            
            eucl_dist <- distance2(x, W %*% H)
            errorx <- mean(abs(x - W %*% H))/mean(x)
            x_pred = W %*% H
            rmsex <- sqrt(mean((x_complet[index] - x_pred[index])^2))
            
            cat("Iter = ", iter, "\t")
            cat("relative error = ", errorx, "\t")
            cat("RMSE", rmsex, "\t")
            cat("diff = ", diff, "\t")
            cat("eucl dist = ", eucl_dist, "\n")
            
            if (errorx < 1e-05) {
                cat("Execution finishes at iteration = ", iter, "\n")
                break
            }
        }
    }
    
    z <- c(list(W = W, H = H))
    z
}

In [32]:
# Non-negative Matrix Factorization via multinomial
nnmf_prob <- function(x, k, maxiter, eps = 100 * .Machine$double.eps, index, x_complet) {
    print_iter <- 50
    powers <- 1.5 + (2.5 - 1.5) * ((1:maxiter) - 1)/(maxiter - 1)
    
    D <- dim(x)[1L]
    N <- dim(x)[2L]
    
    X_factor <- sum(x)
    X_org <- x
    x <- x/X_factor
    
    set.seed(2020) # Reproductibitlité
    W <- matrix(abs(rnorm(D * k)), D, k)
    W <- W/t(matrix(rep(colSums(W), D), ncol(W), nrow(W)))
    set.seed(2020) # Reproductibitlité
    H <- matrix(abs(rnorm(k * N)), k, N)
    H <- H/(matrix(rep(rowSums(H), N), nrow(H), ncol(H)))
    
    P <- matrix(rep(1), k, 1)
    P <- P/sum(P)
    
    W1 <- W
    H1 <- H
    
    Xr_old <- W %*% H
    
    for (iter in 1:maxiter) {
        Qnorm <- (W %*% diag(diag(P), k, k)) %*% H
        
        for (j in 1:k) {
            Q <- (t(t(W[, j])) %*% H[j, ] * P[j])/(Qnorm + eps)
            XQ <- x * Q
            
            dummy <- rowSums(XQ)
            W1[, j] <- t(dummy/sum(dummy))
            
            dummy <- colSums(XQ)
            H1[j, ] <- (dummy/sum(dummy))
            
        }
        W <- W1
        H <- H1
        
        if (iter%%print_iter == 0) {
            Xr <- W %*% H
            diff <- sum(abs(Xr_old - Xr))
            Xr_old <- Xr
            
            eucl_dist <- distance2(x, W %*% H)
            errorx <- mean(abs(x - W %*% H))/mean(x)
            
            x_pred = W %*% H
            rmsex <- sqrt(mean((x_complet[index] - x_pred[index])^2))
            
            cat("Iter = ", iter, "\t")
            cat("relative error = ", errorx, "\t")
            cat("RMSE", rmsex, "\t")
            cat("diff = ", diff, "\t")
            cat("eucl dist = ", eucl_dist, "\n")
            
            if (errorx < 1e-05) {
                cat("Execution finishes at iteration = ", iter, "\n")
                break
            }
        }
    }
    
    W <- W %*% diag(diag(sqrt(P)), k, k) * X_factor
    H <- diag(diag(sqrt(P)), k, k) %*% H
    
    z <- c(list(W = W, H = H))
    z
}

In [33]:
## Fonction pour le choix d'entrainement main function of Non-negative Matrix
## Factorization
nnmf <- function(x, k, method = "nnmf_mm", maxiter = 1000, eps = 2.2204e-16, index, x_complet) {
    if (method == "nnmf_als") {
        cat("Alternating Least Squares Algorithm", "\n")
        nnmf_als(x, k, maxiter, eps, index, x_complet)
        
    } else if (method == "nnmf_prob") {
        cat("Multinomial Algorithm", "\n")
        nnmf_prob(x, k, maxiter, eps, index, x_complet)
        
    } else {
        cat("Multiplicative Update Algorithm", "\n")
        nnmf_mm(x, k, maxiter, eps, index, x_complet)
    }
}

# Entrainement

## Entrainement simple

In [34]:
## Initialisation de la liste pour le tableau comparatif (voir plus bas)
param.tableau <- list()

In [35]:
# Entrainement simple
rang = parameters$rang
algorithme = parameters$algorithme
max_iter = parameters$max_iter

In [36]:
start_time <- Sys.time()  # Temps d'entrainement
entrainement <- nnmf(x = manq.mat, k = rang, method = algorithme, maxiter = max_iter, 
    index = index, x_complet = mat.complet)
end_time <- Sys.time()
temps.entrainement <- format(end_time - start_time, digits = 6)
param.tableau$temps_entrainement <- temps.entrainement

Multiplicative Update Algorithm 
Iter =  50 	relative error =  0.9511352 	RMSE 32.50949 	diff =  14765787 	eucl dist =  542481467 
Iter =  100 	relative error =  0.9429004 	RMSE 32.71938 	diff =  1685077 	eucl dist =  537616354 
Iter =  150 	relative error =  0.9398576 	RMSE 32.80101 	diff =  1094112 	eucl dist =  535646279 
Iter =  200 	relative error =  0.938135 	RMSE 32.85228 	diff =  822560.2 	eucl dist =  534497591 
Iter =  250 	relative error =  0.9370202 	RMSE 32.88894 	diff =  660991.1 	eucl dist =  533729430 
Iter =  300 	relative error =  0.9363044 	RMSE 32.91509 	diff =  529230 	eucl dist =  533196442 
Iter =  350 	relative error =  0.9358283 	RMSE 32.93364 	diff =  435210.2 	eucl dist =  532799354 
Iter =  400 	relative error =  0.9354896 	RMSE 32.94888 	diff =  370891.5 	eucl dist =  532496087 
Iter =  450 	relative error =  0.9352521 	RMSE 32.9614 	diff =  308197.4 	eucl dist =  532278369 
Iter =  500 	relative error =  0.9350941 	RMSE 32.97 	diff =  266262.1 	eucl dist =

In [37]:
## Sortir les matrices et les facteurs latents
U <- entrainement$W
V <- entrainement$H
mat_hat <- U %*% V

In [38]:
dossier.experience <- parameters$dossier_experience
fichier.experience <- parameters$fichier_experience
## Ajouter les paramètres pour identifier le fichier
nom.fichier <- paste0(dossier.experience, fichier.experience,"_r",rang, "-", algorithme, ".mat")
## Sauvegarde du fichier
writeMat(nom.fichier, mat = mat_hat) ## Soit le mettre dans un dossier pour le modèle ou non (?)

# Préparation pour le rapport

## Tableau comparatif

In [39]:
## Nom du modèle
param.tableau$modele <- "FMNN"

## Hyperparamètres utilisés
param.tableau$hyperparametres <- paste("rang:", rang, "| algorithme:", algorithme, sep = " ")

## Calcul du RMSE
param.tableau$rmse <- RMSE(mat.complet, mat_hat, index)

## Calcul du MAPE
param.tableau$mape <- MAPE(mat.complet, mat_hat, index)

## Mettre dans un tableau (ajouter le nom du fichier d'expérience)
param.entrainement <- as.data.frame(list(modele = param.tableau$modele, 
                                         hyperparametres = param.tableau$hyperparametres, 
                                         fichier = fichier.experience, ## Seul paramètre hors liste
                                         rmse = param.tableau$rmse,
                                         mape = param.tableau$mape,
                                         temps = param.tableau$temps_entrainement))

## Mise dans le fichier de l'ensemble des modèles
param.tableau$fichier.comparatif <- "exp/fichier_comparatif.csv"


## Intégrer les données dans le fichier CSV
write.table(param.entrainement, 
            file = param.tableau$fichier.comparatif,
            sep = ",",
            col.names = FALSE, 
            append = TRUE)

## Graphiques : Série temporelle

In [40]:
## Série 1
y1_pred <- mat_hat[1, ]
y1_orig <- mat.complet[1, ]
liste1 <- list(y1_pred, y1_orig)
nom.fichier <- paste0(dossier.experience, fichier.experience,"_r",rang, "-", algorithme, "_L1.json")
write_json(liste1, nom.fichier)

## Série 2
y2_pred <- mat_hat[2, ]
y2_orig <- mat.complet[2, ]
liste2 <- list(y2_pred, y2_orig)
nom.fichier <- paste0(dossier.experience, fichier.experience,"_r",rang, "-", algorithme, "_L2.json")
write_json(liste2, nom.fichier)

## Série 3
y3_pred <- mat_hat[3, ]
y3_orig <- mat.complet[3, ]
liste3 <- list(y3_pred, y3_orig)
nom.fichier <- paste0(dossier.experience, fichier.experience,"_r",rang, "-", algorithme, "_L3.json")
write_json(liste3, nom.fichier)

## Série 4
y4_pred <- mat_hat[4, ]
y4_orig <- mat.complet[4, ]
liste4 <- list(y4_pred, y4_orig)
nom.fichier <- paste0(dossier.experience, fichier.experience,"_r",rang, "-", algorithme, "_L4.json")
write_json(liste4, nom.fichier)

## Graphiques : Comparatif prédits c. originaux

In [41]:
# Matrices pour le rapport (graphique) Graph de comparaison
index_orig <- mat.complet[index]
index_pred <- mat_hat[index]
index_diff <- abs(index_orig - index_pred)

## Mettre sous forme de dataframe
index_df <- as.data.frame(cbind(index_orig, index_pred, index_diff))

## Trouver les quantiles (0 à 1)
q_index <- index_df[, 3]
quantiles_index <- quantile(q_index)
q0 <- quantiles_index[1]
q1 <- quantiles_index[2]
q2 <- quantiles_index[3]
q3 <- quantiles_index[4]
q4 <- quantiles_index[5]

## Donner des catégories (forces grâce aux quantiles)
index_df$dist_axe <- ifelse(index_diff > q0 & index_diff < q1, 1, ifelse(index_diff > 
    q1 & index_diff < q2, 2, ifelse(index_diff > q2 & index_diff < q3, 3, ifelse(index_diff > 
    q3 & index_diff < q4, 4, 5))))

## Sauvegarder le dataframe
index_df <- as.matrix(index_df)

nom.fichier <- paste0(dossier.experience, fichier.experience,"_r",rang, "-", algorithme, "_comparaison.mat")
writeMat(nom.fichier, index_comparaison = index_df)

## Recherche d'hyperparamètres

In [42]:
# Recherche d'hyperparamètres
rech_hyper <- FALSE  # Choisir si la recherche d'hyperparamètres doit être faite


rmse.list <- list()
method.list <- list()

## Entrainement du modèle (recherche d'hyperparamètres) Algorithmes de descente (1
## = ALS, 2 = PROB, 3 = MM)
algo = seq(1, 3, by = 1)

# Séquences de valeurs
rang = seq(5, 30, by = 20)  # k
epsilon = seq(0.1, 1, by = 0.5)  # eps
max_iter = 10  # Maximum d'itérations pour l'ensemble (petit nombre d'itérations pour le moment)


df_hyper = data.frame()  # Pour choisir les hyperparamètres optimaux

if (rech_hyper == TRUE) {
    for (r in rang) {
        for (e in epsilon) {
            for (a in algo) {
                message(a)
                if (a == 1) {
                  algorithme = "Multiplicative"
                  message("-----------------", "\nEntrainement avec : ", "\n\tAlgorithme: ", 
                    algorithme, "\n\tRang: ", r, "\n\tEpsilon: ", e)
                  A = nnmf(x = manq.mat, k = r, method = "nnmf_mm", maxiter = max_iter, 
                    eps = e, index = index, x_complet = mat.complet)
                  # Reformer la matrice de départ
                  U = A$W
                  V = A$H
                  mat_hat <- U %*% V
                  
                  # Calculer le RMSE
                  rmse = RMSE(mat.complet, mat_hat, index)
                  message("\nRMSE: ", rmse)
                  
                  
                } else if (a == 2) {
                  algorithme = "Multinomial"
                  message("-----------------", "\nEntrainement avec : ", "\n\tAlgorithme: ", 
                    algorithme, "\n\tRang: ", r, "\n\tEpsilon: ", e)
                  A = nnmf(x = manq.mat, k = r, method = "nnmf_prob", maxiter = max_iter, 
                    eps = e, index = index, x_complet = mat.complet)
                  # Reformer la matrice de départ
                  U = A$W
                  V = A$H
                  mat_hat <- U %*% V
                  
                  # Calculer le RMSE
                  rmse = RMSE(mat.complet, mat_hat, index)
                  message(" RMSE: ", rmse)
                  
                  
                } else {
                  algorithme = "ALS"
                  message("-----------------", "\nEntrainement avec : ", "\n\tAlgorithme: ", 
                    algorithme, "\n\tRang: ", r, "\n\tEpsilon: ", e)
                  A = nnmf(x = manq.mat, k = r, method = "nnmf_als", maxiter = max_iter, 
                    eps = e, index = index, x_complet = mat.complet)
                  # Reformer la matrice de départ
                  U = A$W
                  V = A$H
                  mat_hat <- U %*% V
                  
                  # Calculer le RMSE
                  rmse = RMSE(mat.complet, mat_hat, index)
                  message(" RMSE: ", rmse)
                  
                }
                liste_hyper <- c(Rang = r, Epsilon = e, Algorithme = a, RMSE = rmse)
                liste_hyper <- t(as.data.frame(liste_hyper))
                df_hyper <- rbind(df_hyper, liste_hyper, make.row.names = FALSE)
                
            }
        }
        
        
        
        
    }
    ################## Réentrainement du meilleur modèle
    
    
    
    
    ## Trouver le modèle avec le plus petit RMSE
    hyper.min <- df_hyper[which.min(df_hyper$RMSE), ]
    hyper.min.list <- c(hyper.min)
    
    ## Sortir les hyperparamètres
    rang = as.integer(as.character(hyper.min.list$Rang))
    epsilon = as.integer(as.character(hyper.min.list$Epsilon))
    algorithme = hyper.min.list$Algorithme
    if (algorithme == 1) {
        algo = "nnmf_mm"
    } else if (algorithme == 2) {
        algo = "nnmf_prob"
    } else {
        algo = "nnmf_als"
    }
    
    
    ## Entrainement du modèle
    start_time <- Sys.time()  # Temps d'entrainement
    entrainement <- nnmf(x = manq.mat, k = rang, method = algo, maxiter = max_iter, 
        eps = epsilon, index = index, x_complet = mat.complet)
    end_time <- Sys.time()
    temps.entrainement <- format(end_time - start_time, digits = 6)
    
    ## Créer la matrice de valeurs prédites
    U <- entrainement$W
    V <- entrainement$H
    mat_hat <- U %*% V
    
    ## Sauvegarder la matrice avec les éléments prédits
    message("\nSauvegarde de la matrice prédite")
    dossier_pred <- paste0("exp/", dossier)
    fichier.entrainement <- paste0(fichier.binaire, "_FMNN-hyper")
    writeMat(paste0(dossier_pred, fichier.entrainement, ".mat"), mat = mat_hat)
    
    
    ############################### PRÉPARATION POUR LE RAPPORT
    
    
    ## Préparation pour le tableau comparatif
    modele <- "FMNN"
    hyperparametres <- paste("rang:", rang, "| epsilon:", epsilon, "| algorithme:", 
        algorithme, sep = " ")
    fichier.util <- fichier.binaire
    rmse = RMSE(mat.complet, mat_hat, index)
    mape = MAPE(mat.complet, mat_hat, index)
    
    ## Mettre les informations d'entrainement dans un fichier CSV
    param.entrainement <- as.data.frame(list(modele = modele, hyperparametres = hyperparametres, 
        fichier = fichier.util, rmse = rmse, mape = mape, temps = temps.entrainement))
    fichier.comparatif <- paste0("exp/", "fichier_comparatif.csv")
    write.table(param.entrainement, file = fichier.comparatif, sep = ",", col.names = FALSE, 
        append = TRUE)
    
    
    
    ## Préparation pour les graphiques
    
    ## Graph série temporelle
    y1_pred <- mat_hat[1, ]
    y1_orig <- mat.complet[1, ]
    liste1 <- list(y1_pred, y1_orig)
    ## Enregistrer la liste
    nom.fichier <- paste0("exp/", dossier, "ligne1_", fichier.entrainement, ".json")
    write_json(liste1, nom.fichier)
    
    y2_pred <- mat_hat[2, ]
    y2_orig <- mat.complet[2, ]
    liste2 <- list(y2_pred, y2_orig)
    ## Enregistrer la liste
    nom.fichier <- paste0("exp/", dossier, "ligne2_", fichier.entrainement, ".json")
    write_json(liste2, nom.fichier)
    
    
    y3_pred <- mat_hat[3, ]
    y3_orig <- mat.complet[3, ]
    liste3 <- list(y3_pred, y3_orig)
    ## Enregistrer la liste
    nom.fichier <- paste0("exp/", dossier, "ligne3_", fichier.entrainement, ".json")
    write_json(liste3, nom.fichier)
    
    y4_pred <- mat_hat[4, ]
    y4_orig <- mat.complet[4, ]
    liste4 <- list(y4_pred, y4_orig)
    ## Enregistrer la liste
    nom.fichier <- paste0("exp/", dossier, "ligne4_", fichier.entrainement, ".json")
    write_json(liste4, nom.fichier)
    
    # Matrices pour le rapport (graphique) Graph de comparaison
    index_orig <- mat.complet[index]
    index_pred <- mat_hat[index]
    index_diff <- abs(index_orig - index_pred)
    
    ## Mettre sous forme de dataframe
    index_df <- as.data.frame(cbind(index_orig, index_pred, index_diff))
    
    ## Trouver les quantiles (0 à 1)
    q_index <- index_df[, 3]
    quantiles_index <- quantile(q_index)
    q0 <- quantiles_index[1]
    q1 <- quantiles_index[2]
    q2 <- quantiles_index[3]
    q3 <- quantiles_index[4]
    q4 <- quantiles_index[5]
    
    ## Donner des catégories (forces grâce aux quantiles)
    index_df$dist_axe <- ifelse(index_diff > q0 & index_diff < q1, 1, ifelse(index_diff > 
        q1 & index_diff < q2, 2, ifelse(index_diff > q2 & index_diff < q3, 3, ifelse(index_diff > 
        q3 & index_diff < q4, 4, 5))))
    
    ## Sauvegarder le dataframe
    index_df <- as.matrix(index_df)
    
    nom.fichier <- paste0("exp/", dossier, "compar_", fichier.entrainement, "_index-comparaison.mat")
    writeMat(nom.fichier, index_comparaison = index_df)
    
    
    
    
}